In [1]:
from selenium.webdriver.common.by import By
import selenium as sel
import pandas as pd
import time
import datetime
import re

import base as bs

import os as os
import csv

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [3]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.semana.com/buscador/?query={empresa}')
time.sleep(2)

In [4]:
# Extrae la lista de todos los articulos de la pagina
articulos = driver.find_elements(By.XPATH,'.//div[contains(@class,"queryly_item_row")]')

In [5]:
# Itera por cada articulo y extrae la informacion (EN CASO DE QUE NO EXISTA ARCHIVO DE ALMACENAMIENTO ANTERIOR)
titulares = []
for art in articulos:
    url = art.find_element(By.XPATH,'.//a').get_attribute('href')
    fechaP = art.find_element(By.XPATH,'.//div[contains(@style,"margin-bottom:0px;color:#555;font-size:12px;")]').text
    resumen = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_description")]').text
    titulo = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_title")]').text
    txtImage = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_advanced_item_imagecontainer")]').get_attribute('style')
    imagen = 'https://www.semana.com'
    imagen = imagen + txtImage.split("\"")[1]
    titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    'Titulo':titulo,
                    'Fecha Publicacion':fechaP,
                    'Resumen':resumen,
                    'URL':url,
                    'Imagen':imagen,
                    'Empresa':empresa})
        
    

In [6]:
# Itera por cada articulo y extrae la informacion (CASO DE QUE YA EXISTA ARCHIVO DONDE SE ALMACENA)
titulares = []
for art in articulos:
    url = art.find_element(By.XPATH,'.//a').get_attribute('href')
    if not(bs.existedb(url, "semana")):
        fechaP = art.find_element(By.XPATH,'.//div[contains(@style,"margin-bottom:0px;color:#555;font-size:12px;")]').text
        resumen = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_description")]').text
        titulo = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_item_title")]').text
        txtImage = art.find_element(By.XPATH,'.//div[contains(@class,"queryly_advanced_item_imagecontainer")]').get_attribute('style')
        imagen = 'https://www.semana.com'
        imagen = imagen + txtImage.split("\"")[1]
        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Titulo':titulo,
                        'Fecha Publicacion':fechaP,
                        'Resumen':resumen,
                        'URL':url,
                        'Imagen':imagen,
                        'Empresa':empresa})

In [ ]:
# busca los autores de cada articulo y las almacena en la lista de titulares
for tit in titulares:
    
    driver.get(tit['URL'])

    # agregar contenido al dict de titulares
    tit['Contenido'] = bs.obtener_contenido(driver)

    # agregar tags al dict de titulares
    tit['Tags'] = bs.obtener_tags(driver)
    
    # agregar contenido al dict de titulares
    tit['Tema'] = bs.obtener_tema(driver)


In [8]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Imagen','Empresa','Contenido','Tags','Tema']

In [12]:
# Para AGREGAR titulares a un archivo.
with open('../data/raw/semana.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)


{'Fecha Extraccion': '2023-01-26 21:46:36', 'Titulo': 'BVC se mantuvo a la baja este martes mientras que Ecopetrol cerró con saldo a ...', 'Fecha Publicacion': 'mar, 17 ene 2023', 'Resumen': 'La Bolsa de Valores de Colombia y Ecopetrol terminaron con resultados dispares este martes -17 de enero- luego de una sesión de altas y bajas ...', 'URL': 'https://www.semana.com/economia/inversionistas/articulo/bvc-se-mantuvo-a-la-baja-este-martes-mientras-que-ecopetrol-cerro-con-saldo-a-favor/202302/', 'Imagen': 'https://www.semana.com/resizer/60ckNqED2bNd-1kIvLwLBMxQVkE=/300x0/arc-anglerfish-arc2-prod-semana/public/JHA37X2KZFHOVF3M4VXCQEU5EM.jpg', 'Empresa': 'ecopetrol', 'Contenido': 'La Bolsa de Valores de Colombia y Ecopetrol terminaron con resultados dispares este martes -17 de enero- luego de una sesión de altas y bajas muy marcadas en el mercado local, donde se mantiene la incertidumbre por lo que pasará con el país en materia inflacionaria y lo que pasará con las tasas de interés la próxi

In [10]:
#Para crear un NUEVO archivo con los titulares
#pd.json_normalize(titulares).to_csv(f'../data/raw/semana.csv',index=False, encoding='latin-1', errors='ignore', columns=columnas)